In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated, Dict, Any, List, Optional
from langchain_core.messages import BaseMessage, HumanMessage,SystemMessage,AIMessage
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from datetime import date
from enum import Enum

In [2]:
load_dotenv()

True

In [3]:
llm = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0.7
)

In [4]:
class GoalCreate(BaseModel):
    goal_name: str = Field(..., max_length=200)
    description: Optional[str] = None

    target_date: Optional[date] = None

    importance_level: int = Field(default=1, ge=1, le=5)
    motivations: Optional[List[str]] = None

In [5]:
goal_prompt_llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.1   # low temp = deterministic normalization
)

goal_prompt_structured_llm = goal_prompt_llm.with_structured_output(
    GoalCreate
)

In [6]:
# backend/app/ai/prompts/goal_prompt_builder.py

GOAL_PROMPT_SYSTEM = """
You are a goal normalization engine.

Your task:
- Convert informal user input into a structured goal.

Rules:
- goal_name must be concise and specific
- description should expand what the user wants to achieve
- target_date:
  - infer ONLY if user explicitly mentions a time (e.g. "2022-12-25") use current time as reference if user mentions duration to evaluate target date
  - otherwise set it to null
- importance_level:
  - infer urgency (1 = casual, 5 = very important)
- motivations:
  - infer clear motivations if stated or obvious
  - otherwise null

Do NOT ask questions.
Do NOT add extra fields.
Return ONLY structured output.
"""


In [ ]:
# backend/app/ai/state.py


from langgraph.graph.message import add_messages



class ChatState(TypedDict, total=False):
    # conversation memory
    messages: Annotated[List[BaseMessage], add_messages]

    # intent decided by orchestrator
    intent: str

    # routine generator output
    routine_proposal: Dict[str, Any]

    # evaluator output (structured)
    evaluation: Dict[str, Any]

    # loop / control info
    iteration: int

    max_iterations: int


In [8]:
# backend/app/ai/schemas/routine.py


class SubtaskType(str, Enum):
    checkbox = "checkbox"
    count = "count"
    duration = "duration"
    score = "score"


class GeneratedSubtask(BaseModel):
    temp_subtask_key: str

    subtask_name: str
    subtask_type: SubtaskType

    target_value: Optional[float] = None
    weight: int = 1
    deadline: Optional[date] = None

    depends_on_subtask_key: Optional[str] = None


class GeneratedTask(BaseModel):
    temp_task_key: str

    task_name: str
    description: Optional[str] = None
    difficulty: int = Field(ge=1, le=5)

    depends_on_task_key: Optional[str] = None
    subtasks: List[GeneratedSubtask]


class RoutineGenerationOutput(BaseModel):
    tasks: List[GeneratedTask]


In [9]:
class RoutineSuggestions(BaseModel):
    suggestions: List[str]

In [10]:
class RoutineLLMOutput(BaseModel):
    routine: RoutineGenerationOutput
    suggestions: RoutineSuggestions

In [11]:
routine_llm = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0.3
)
routine_structured_llm = routine_llm.with_structured_output(
    RoutineLLMOutput
)

In [12]:
class ChatState(TypedDict, total=False):
    messages: Annotated[List[BaseMessage], add_messages]

    intent: str

    structured_goal: Dict[str, Any]   # GoalCreate output

    routine_tasks: List[Dict[str, Any]]

In [13]:
ROUTINE_SYSTEM_PROMPT = """
You are a routine generation engine.

Generate:
1. Tasks and subtasks ONLY in structured form
2. High-level suggestions for the user as text

Rules:
- Tasks must match Task + Subtask schema
- No IDs, no completion fields
- Use temporary keys for dependencies
"""


def routine_generator_node(
    state: ChatState
) -> ChatState:
    
    goal = state["messages"][-1].content  # user goal text

    messages = [
        SystemMessage(content=ROUTINE_SYSTEM_PROMPT),
        HumanMessage(content=goal)
    ]
    print("Messages to routine LLM:", messages)
    llm_output = routine_structured_llm.invoke(messages)
    print("Routine LLM output:", llm_output)
    # 1️⃣ Structured routine → state
    new_state = {
        **state,
        "routine_tasks": llm_output.routine.tasks
    }

    # 2️⃣ Suggestions → chat message
    suggestion_text = "\n".join(
        f"- {s}" for s in llm_output.suggestions.suggestions
    )

    new_state["messages"] = [
        AIMessage(
            content=f"Here are some suggestions to follow along with this routine:\n{suggestion_text}"
        )
    ]

    return new_state

In [14]:
def goal_prompt_builder_node(state:ChatState
) -> ChatState:
    """
    Converts user input into GoalCreate schema.
    """
    print("Routine generator node input state")
    user_text = state["messages"][-1].content

    messages = [
        SystemMessage(content=GOAL_PROMPT_SYSTEM),
        HumanMessage(content=user_text),
    ]
    print("Messages to goal LLM:", messages)
    goal: GoalCreate = goal_prompt_structured_llm.invoke(messages)
    print("Goal LLM output:", goal)
    # update state with structured goal
    new_state = {
        **state,
        "structured_goal": goal.model_dump()
    }

    # optional: confirmation message for UI
    confirmation = (
        "I’ve understood your goal as:\n\n"
        f"🎯 Goal: {goal.goal_name}\n"
        f"📝 Description: {goal.description or '—'}\n"
        f"📅 Target Date: {goal.target_date or 'Not specified'}\n"
        f"⭐ Importance Level: {goal.importance_level}/5\n"
    )

    if goal.motivations:
        confirmation += "💡 Motivations:\n"
        for m in goal.motivations:
            confirmation += f"• {m}\n"

    new_state["messages"] = [
        AIMessage(content=confirmation)
    ]

    return new_state

In [15]:
graph = StateGraph(ChatState)

# add nodes
graph.add_node('routine_generator_node', routine_generator_node)
graph.add_node('goal_prompt_builder_node', goal_prompt_builder_node)

graph.add_edge(START, 'goal_prompt_builder_node')
graph.add_edge('goal_prompt_builder_node', 'routine_generator_node')
graph.add_edge('routine_generator_node', END)

chatbot = graph.compile()

In [16]:
initial_state = {
    'messages': [HumanMessage(content='i want to complete dsa from basics to advance level in 6 months')]
}

chatbot.invoke(initial_state)

Routine generator node input state
Messages to goal LLM: [SystemMessage(content='\nYou are a goal normalization engine.\n\nYour task:\n- Convert informal user input into a structured goal.\n\nRules:\n- goal_name must be concise and specific\n- description should expand what the user wants to achieve\n- target_date:\n  - infer ONLY if user explicitly mentions a time (e.g. "2022-12-25") use current time as reference if user mentions duration to evaluate target date\n  - otherwise set it to null\n- importance_level:\n  - infer urgency (1 = casual, 5 = very important)\n- motivations:\n  - infer clear motivations if stated or obvious\n  - otherwise null\n\nDo NOT ask questions.\nDo NOT add extra fields.\nReturn ONLY structured output.\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='i want to complete dsa from basics to advance level in 6 months', additional_kwargs={}, response_metadata={})]
Goal LLM output: goal_name='DSA' description='Complete DSA from basics to adva

{'messages': [HumanMessage(content='i want to complete dsa from basics to advance level in 6 months', additional_kwargs={}, response_metadata={}, id='9ab3dc87-e768-4243-94e2-c3e7ca2811ab'),
  AIMessage(content='I’ve understood your goal as:\n\n🎯 Goal: DSA\n📝 Description: Complete DSA from basics to advanced level\n📅 Target Date: 2023-06-01\n⭐ Importance Level: 5/5\n', additional_kwargs={}, response_metadata={}, id='7ad3a62f-861e-45e4-ac19-e93cbdc6a865'),
  AIMessage(content='Here are some suggestions to follow along with this routine:\n- Allocate a consistent daily study block (e.g., 1‑2 hours) and stick to it.\n- Use spaced repetition for theory concepts and flashcards for algorithmic patterns.\n- Practice coding on platforms like LeetCode, Codeforces, or HackerRank after each subtask.\n- Schedule weekly review sessions to revisit solved problems and identify gaps.\n- Before each contest, simulate the environment: set a timer, avoid distractions, and review common pitfalls.', addition

In [17]:
Messages to routine LLM: [SystemMessage(content='\nYou are a routine generation engine.\n\nGenerate:\n1. Tasks and subtasks ONLY in structured form\n2. High-level suggestions for the user as text\n\nRules:\n- Tasks must match Task + Subtask schema\n- No IDs, no completion fields\n- Use temporary keys for dependencies\n- Suggestions must be general advice, not instructions\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='i want to complete dsa from basics to advance level in 6 months', additional_kwargs={}, response_metadata={})]
Routine LLM output: routine=RoutineGenerationOutput(tasks=[GeneratedTask(temp_task_key='t1', task_name='Foundations of Programming & Basic Algorithms', description='Cover programming fundamentals, complexity analysis, and simple algorithmic patterns.', difficulty=2, depends_on_task_key=None, subtasks=[GeneratedSubtask(temp_subtask_key='t1_s1', subtask_name='Read introductory chapters on variables, control flow, and functions', subtask_type=<SubtaskType.duration: 'duration'>, target_value=30.0, weight=1, deadline=datetime.date(2026, 2, 7), depends_on_subtask_key=None), GeneratedSubtask(temp_subtask_key='t1_s2', subtask_name='Solve 100 easy coding problems (e.g., arrays, loops)', subtask_type=<SubtaskType.count: 'count'>, target_value=100.0, weight=1, deadline=datetime.date(2026, 2, 28), depends_on_subtask_key='t1_s1'), GeneratedSubtask(temp_subtask_key='t1_s3', subtask_name='Complete weekly quizzes (4 total) with ≥70% score', subtask_type=<SubtaskType.score: 'score'>, target_value=70.0, weight=1, deadline=datetime.date(2026, 2, 28), depends_on_subtask_key='t1_s2')]), GeneratedTask(temp_task_key='t2', task_name='Core Data Structures – Linear', description='Learn and practice arrays, linked lists, stacks, and queues.', difficulty=3, depends_on_task_key='t1', subtasks=[GeneratedSubtask(temp_subtask_key='t2_s1', subtask_name='Watch video lectures on arrays and linked lists', subtask_type=<SubtaskType.duration: 'duration'>, target_value=20.0, weight=1, deadline=datetime.date(2026, 3, 7), depends_on_subtask_key=None), GeneratedSubtask(temp_subtask_key='t2_s2', subtask_name='Implement each structure from scratch in your preferred language', subtask_type=<SubtaskType.count: 'count'>, target_value=4.0, weight=1, deadline=datetime.date(2026, 3, 14), depends_on_subtask_key='t2_s1'), GeneratedSubtask(temp_subtask_key='t2_s3', subtask_name='Solve 80 medium‑difficulty problems focusing on these structures', subtask_type=<SubtaskType.count: 'count'>, target_value=80.0, weight=1, deadline=datetime.date(2026, 3, 31), depends_on_subtask_key='t2_s2'), GeneratedSubtask(temp_subtask_key='t2_s4', subtask_name='Pass a short assessment with ≥75% score', subtask_type=<SubtaskType.score: 'score'>, target_value=75.0, weight=1, deadline=datetime.date(2026, 3, 31), depends_on_subtask_key='t2_s3')]), GeneratedTask(temp_task_key='t3', task_name='Recursion, Sorting & Searching', description='Master recursion patterns and classic sorting/searching algorithms.', difficulty=3, depends_on_task_key='t2', subtasks=[GeneratedSubtask(temp_subtask_key='t3_s1', subtask_name='Study recursion concepts and common patterns (divide‑and‑conquer)', subtask_type=<SubtaskType.duration: 'duration'>, target_value=15.0, weight=1, deadline=datetime.date(2026, 4, 7), depends_on_subtask_key=None), GeneratedSubtask(temp_subtask_key='t3_s2', subtask_name='Implement and benchmark Bubble, Insertion, Merge, Quick, and Heap sort', subtask_type=<SubtaskType.count: 'count'>, target_value=5.0, weight=1, deadline=datetime.date(2026, 4, 14), depends_on_subtask_key='t3_s1'), GeneratedSubtask(temp_subtask_key='t3_s3', subtask_name='Solve 70 problems involving recursion, sorting, and binary search', subtask_type=<SubtaskType.count: 'count'>, target_value=70.0, weight=1, deadline=datetime.date(2026, 4, 30), depends_on_subtask_key='t3_s2'), GeneratedSubtask(temp_subtask_key='t3_s4', subtask_name='Complete a timed quiz (30\u202fmin) with ≥80% correct answers', subtask_type=<SubtaskType.score: 'score'>, target_value=80.0, weight=1, deadline=datetime.date(2026, 4, 30), depends_on_subtask_key='t3_s3')]), GeneratedTask(temp_task_key='t4', task_name='Advanced Data Structures – Trees & Graphs', description='Deep dive into binary trees, BSTs, AVL/Red‑Black trees, heaps, and graph representations.', difficulty=4, depends_on_task_key='t3', subtasks=[GeneratedSubtask(temp_subtask_key='t4_s1', subtask_name='Read and summarize chapters on trees and graph fundamentals', subtask_type=<SubtaskType.duration: 'duration'>, target_value=25.0, weight=1, deadline=datetime.date(2026, 5, 7), depends_on_subtask_key=None), GeneratedSubtask(temp_subtask_key='t4_s2', subtask_name='Implement traversal algorithms (in‑order, pre‑order, post‑order, BFS, DFS)', subtask_type=<SubtaskType.count: 'count'>, target_value=5.0, weight=1, deadline=datetime.date(2026, 5, 14), depends_on_subtask_key='t4_s1'), GeneratedSubtask(temp_subtask_key='t4_s3', subtask_name='Solve 90 problems covering tree operations and basic graph algorithms', subtask_type=<SubtaskType.count: 'count'>, target_value=90.0, weight=1, deadline=datetime.date(2026, 5, 31), depends_on_subtask_key='t4_s2'), GeneratedSubtask(temp_subtask_key='t4_s4', subtask_name='Pass a mini‑project: build a simple path‑finding visualizer', subtask_type=<SubtaskType.checkbox: 'checkbox'>, target_value=None, weight=1, deadline=datetime.date(2026, 5, 31), depends_on_subtask_key='t4_s3')]), GeneratedTask(temp_task_key='t5', task_name='Algorithmic Paradigms – DP, Greedy, Backtracking', description='Learn dynamic programming, greedy strategies, and backtracking techniques.', difficulty=4, depends_on_task_key='t4', subtasks=[GeneratedSubtask(temp_subtask_key='t5_s1', subtask_name='Watch tutorial series on DP, greedy, and backtracking', subtask_type=<SubtaskType.duration: 'duration'>, target_value=20.0, weight=1, deadline=datetime.date(2026, 6, 7), depends_on_subtask_key=None), GeneratedSubtask(temp_subtask_key='t5_s2', subtask_name='Create a cheat‑sheet of state‑definition patterns for DP', subtask_type=<SubtaskType.checkbox: 'checkbox'>, target_value=None, weight=1, deadline=datetime.date(2026, 6, 10), depends_on_subtask_key=None), GeneratedSubtask(temp_subtask_key='t5_s3', subtask_name='Solve 100 mixed‑difficulty problems focusing on these paradigms', subtask_type=<SubtaskType.count: 'count'>, target_value=100.0, weight=1, deadline=datetime.date(2026, 6, 30), depends_on_subtask_key='t5_s2'), GeneratedSubtask(temp_subtask_key='t5_s4', subtask_name='Take a comprehensive test (90\u202fmin) with ≥75% score', subtask_type=<SubtaskType.score: 'score'>, target_value=75.0, weight=1, deadline=datetime.date(2026, 6, 30), depends_on_subtask_key='t5_s3')]), GeneratedTask(temp_task_key='t6', task_name='Mock Interviews, Review & Polish', description='Simulate interview conditions, identify gaps, and reinforce weak topics.', difficulty=5, depends_on_task_key='t5', subtasks=[GeneratedSubtask(temp_subtask_key='t6_s1', subtask_name='Complete 5 full‑length mock interview sessions', subtask_type=<SubtaskType.count: 'count'>, target_value=5.0, weight=1, deadline=datetime.date(2026, 7, 15), depends_on_subtask_key=None), GeneratedSubtask(temp_subtask_key='t6_s2', subtask_name='Review each mock, note recurring weak areas', subtask_type=<SubtaskType.checkbox: 'checkbox'>, target_value=None, weight=1, deadline=datetime.date(2026, 7, 20), depends_on_subtask_key='t6_s1'), GeneratedSubtask(temp_subtask_key='t6_s3', subtask_name='Re‑solve 30 previously missed problems', subtask_type=<SubtaskType.count: 'count'>, target_value=30.0, weight=1, deadline=datetime.date(2026, 7, 27), depends_on_subtask_key='t6_s2'), GeneratedSubtask(temp_subtask_key='t6_s4', subtask_name='Finalize a personal DSA reference guide', subtask_type=<SubtaskType.checkbox: 'checkbox'>, target_value=None, weight=1, deadline=datetime.date(2026, 7, 31), depends_on_subtask_key='t6_s3')])]) suggestions=RoutineSuggestions(suggestions=['Maintain a consistent daily study routine and allocate focused blocks for theory and practice.', 'Use spaced repetition for key concepts and algorithmic patterns to improve long‑term retention.', 'Track your progress visually (e.g., a Kanban board) to stay motivated and quickly spot bottlenecks.', 'When stuck on a problem, first attempt a paper‑pencil solution before consulting hints or solutions.', 'Periodically revisit earlier topics to reinforce foundational knowledge and prevent forgetting.'])

SyntaxError: invalid syntax (2685954503.py, line 1)